In [ ]:
import os
import sys
import time

import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Dict, Box

from Agents.agent import Agent
# from Agents.fov_points import get_fov_points
from Agents.overlap_detection import detect_overlapping_points
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, FOV_RADIUS
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls

In [ ]:
class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        # defining the observation and action spaces for all the agents
        self.observation_space = Dict({
            'predator_position': Box(low=np.array([0, 0], dtype=np.float32),
                                     high=np.array([SCREEN_WIDTH, SCREEN_HEIGHT], dtype=np.float32),
                                     dtype=np.float32),
            'vision': Dict({
                'vision_points': Discrete(2)
            }),
        })

        # defining the action space based on total number of predator and prey
        # since we are training only one agent so, defining only the necessary number of actions
        self.action_space = Discrete(5)
        # 5 for rotate
        # clockwise, anti-clock
        # move front, move back and wait

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        # the pygame window should be initialized in the render function
        # initializing the pygame
        pygame.init()

        # setting the screen size
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        # initializing the font
        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        # for the wall initializations
        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)

        self.predator_agent = predator_agents

    def _get_obs(self):
        observation = {
            'predator_position': self.predator_agent.current_position,
            'predator_angle': self.predator_agent.angle,
            'vision': detect_overlapping_points(self.predator_agent.current_position, WALLS),
        }

        return observation

    def _max_right(self):
        max_right = 0

        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    # the usual reset function
    def reset(self, seed=0):
        self.start_time = time.time()

        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS)

        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent

        # for predator in self.predator_agents:
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)
        # observation.append([predator.index, predator.agent, predator.current_position])

        # setting the predator and prey to their initial position

        self.predator_agent = predator


        # all the variable values inside the observation space needs to be sent inside the observation variable
        # for this level purpose we decided to add the dictionary observation
        # set the observation to a dictionary
        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        # initializing the return variables
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        # handles the pygame window event when closing
        # !if the window still crashes pygame.event needs to be managed properly
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        # observation needs to be set a dictionary

        self.total_steps += 1
        print(self._max_right())
        # for wall in self.walls:
        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            done = True
        """
        here lies the most important task
        handling the rewards
        """
        reward += 0.01
        self.render()

        # it will update the total reward every step
        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen

            screen.fill(WHITE)
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in WALLS.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))

            pygame.display.update()

    def close(self):
        pygame.quit()

In [ ]:
env = GameEnv()

In [ ]:
from stable_baselines3 import PPO

In [ ]:
log_path = os.path.join('Training', 'Logs', 'Level_01_PPO')
baseline_path = os.path.join('Training', 'Models', 'Level_01_PPO')

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
env.reset()
model.learn(total_timesteps=500000)

In [ ]:
env.close()

In [ ]:
# from Constants.constants import SCREEN_HEIGHT, SCREEN_WIDTH, FOV_RADIUS
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600

FOV_RADIUS = 100
PREDATOR_RADIUS = 20

WALLS = {
    '1': {'x': SCREEN_WIDTH // 2 - 25, 'y': 0, 'width': 50, 'height': 250},
    '2': {'x': SCREEN_WIDTH // 2 - 25, 'y': 350, 'width': 50, 'height': 250}
}


In [ ]:
import tkinter as tk
import random
import math

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 400
PREDATOR_RADIUS = 10
FOV_RADIUS = 150

# Initialize the tkinter window
root = tk.Tk()
root.title("Agent Visualization")

canvas = tk.Canvas(root, width=SCREEN_WIDTH, height=SCREEN_HEIGHT)
canvas.pack()

predator_x = SCREEN_WIDTH // 2
predator_y = SCREEN_HEIGHT // 2

def move_predator(action):
    global predator_x, predator_y
    if action == 0:  # Move up
        predator_y -= 2
    elif action == 1:  # Move down
        predator_y += 2
    elif action == 2:  # Move left
        predator_x -= 2
    elif action == 3:  # Move right
        predator_x += 2

    predator_x = max(PREDATOR_RADIUS, min(SCREEN_WIDTH - PREDATOR_RADIUS, predator_x))
    predator_y = max(PREDATOR_RADIUS, min(SCREEN_HEIGHT - PREDATOR_RADIUS, predator_y))

def render(show_coordinates=True):
    canvas.delete("all")  # Clear the canvas

    # Draw FOV border
    canvas.create_oval(predator_x - FOV_RADIUS, predator_y - FOV_RADIUS,
                       predator_x + FOV_RADIUS, predator_y + FOV_RADIUS,
                       outline="blue")

    # Draw coordinates within FOV
    if show_coordinates:
        for x in range(int(predator_x - FOV_RADIUS), int(predator_x + FOV_RADIUS), 50):
            for y in range(int(predator_y + FOV_RADIUS), int(predator_y - FOV_RADIUS), -50):
                if math.sqrt((x - predator_x) ** 2 + (y - predator_y) ** 2) <= FOV_RADIUS:
                    # Convert coordinates to the desired system
                    x_coord = x - predator_x
                    y_coord = y - predator_y
                    canvas.create_text(x, y, text=f"({x_coord},{y_coord})", fill="black")

    # Draw the agent
    canvas.create_oval(predator_x - PREDATOR_RADIUS, predator_y - PREDATOR_RADIUS,
                       predator_x + PREDATOR_RADIUS, predator_y + PREDATOR_RADIUS,
                       outline="red", fill="red")

def main():
    running = True

    while running:
        action = random.choice([0, 1, 2, 3])
        move_predator(action)

        render(show_coordinates=True)  # Set show_coordinates to True to display coordinates

        root.update()  # Update the tkinter window

    root.quit()

if __name__ == "__main__":
    main()


In [ ]:
root.quit()

new

In [75]:
import tkinter as tk
import random
import math

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800
PREDATOR_RADIUS = 15
FOV_RADIUS = 300

# Initialize the tkinter window
root = tk.Tk()
root.title("Agent Visualization")

canvas = tk.Canvas(root, width=SCREEN_WIDTH, height=SCREEN_HEIGHT)
canvas.pack()

predator_x = 40
predator_y = 40
def move_predator(action):
    global predator_x, predator_y
    if action == 0:  # Move up
        predator_y -= 2
    elif action == 1:  # Move down
        predator_y += 2
    elif action == 2:  # Move left
        predator_x -= 2
    elif action == 3:  # Move right
        predator_x += 2

    predator_x = max(PREDATOR_RADIUS, min(SCREEN_WIDTH - PREDATOR_RADIUS, predator_x))
    predator_y = max(PREDATOR_RADIUS, min(SCREEN_HEIGHT - PREDATOR_RADIUS, predator_y))

def get_fov_grid():
    fov_grid = []

    for x_coord in range(0, SCREEN_WIDTH, 50):
        for y_coord in range(0, SCREEN_HEIGHT, 50):
            if math.sqrt((x_coord - predator_x) ** 2 + (y_coord - predator_y) ** 2) <= FOV_RADIUS:
                fov_grid.append((x_coord, y_coord, 0))  # Mark coordinates inside FOV as 0
            else:
                fov_grid.append((x_coord, y_coord, 1))  # Mark coordinates outside FOV as 1
    return fov_grid

def render(show_coordinates=True):
    canvas.delete("all")  # Clear the canvas

    # Draw FOV border
    canvas.create_oval(predator_x - FOV_RADIUS, predator_y - FOV_RADIUS,
                       predator_x + FOV_RADIUS, predator_y + FOV_RADIUS,
                       outline="blue")

    # Draw coordinates within FOV using your get_fov_grid function
    fov_grid = get_fov_grid()
    for x, y, value in fov_grid:
        if show_coordinates:
            if value == 0:
                canvas.create_text(x, y, text="0", fill="black")
            else:
                x_coord = x - predator_x
                y_coord = y - predator_y
                canvas.create_text(x, y, text=f"({x_coord},{y_coord})", fill="black")

    # Draw the agent
    canvas.create_oval(predator_x - PREDATOR_RADIUS, predator_y - PREDATOR_RADIUS,
                       predator_x + PREDATOR_RADIUS, predator_y + PREDATOR_RADIUS,
                       outline="red", fill="red")

def main():
    running = True

    while running:
        action = random.choice([0, 1, 2, 3])
        move_predator(action)

        render(show_coordinates=True)  # Set show_coordinates to True to display coordinates

        root.update()  # Update the tkinter window

    root.quit()

if __name__ == "__main__":
    main()


TclError: invalid command name ".!canvas"

In [ ]:
root.quit()


In [ ]:
import tkinter as tk
import random
import math

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800
PREDATOR_RADIUS = 10
FOV_RADIUS = 300

# Initialize the tkinter window
root = tk.Tk()
root.title("Agent Visualization")

canvas = tk.Canvas(root, width=SCREEN_WIDTH, height=SCREEN_HEIGHT)
canvas.pack()

predator_x = 40
predator_y = 40

def move_predator(action):
    global predator_x, predator_y
    if action == 0:  # Move up
        predator_y -= 2
    elif action == 1:  # Move down
        predator_y += 2
    elif action == 2:  # Move left
        predator_x -= 2
    elif action == 3:  # Move right
        predator_x += 2

    predator_x = max(PREDATOR_RADIUS, min(SCREEN_WIDTH - PREDATOR_RADIUS, predator_x))
    predator_y = max(PREDATOR_RADIUS, min(SCREEN_HEIGHT - PREDATOR_RADIUS, predator_y))

def get_fov_points(agent_position):
    fov_points = {}
    x, y = agent_position
    for angle in range(360):
        radians = math.radians(angle)
        x_coord = int(x + FOV_RADIUS * math.cos(radians))
        y_coord = int(y + FOV_RADIUS * math.sin(radians))
        if 0 <= x_coord < SCREEN_WIDTH and 0 <= y_coord < SCREEN_HEIGHT:
            fov_points[(x_coord, y_coord)] = 0
    return fov_points

def render(show_coordinates=True):
    canvas.delete("all")  # Clear the canvas

    # Draw FOV border
    canvas.create_oval(predator_x - FOV_RADIUS, predator_y - FOV_RADIUS,
                       predator_x + FOV_RADIUS, predator_y + FOV_RADIUS,
                       outline="blue")

    # Use your get_fov_points function to get the coordinates
    fov_points = get_fov_points((predator_x, predator_y))
    for x, y in fov_points:
        if show_coordinates:
            x_coord = x - predator_x
            y_coord = y - predator_y
            if x_coord == 0 and y_coord == 0:
                canvas.create_text(x, y, text="0", fill="black")
            else:
                canvas.create_text(x, y, text=f"({x_coord},{y_coord})", fill="black")

    # Draw the agent
    canvas.create_oval(predator_x - PREDATOR_RADIUS, predator_y - PREDATOR_RADIUS,
                       predator_x + PREDATOR_RADIUS, predator_y + PREDATOR_RADIUS,
                       outline="red", fill="red")

def main():
    running = True

    while running:
        action = random.choice([0, 1, 2, 3])
        move_predator(action)

        render(show_coordinates=True)  # Set show_coordinates to True to display coordinates

        root.update()  # Update the tkinter window

    root.quit()

if __name__ == "__main__":
    main()


In [ ]:
pygame.init()

predator_x = 40
predator_y = 40
render()